In [2]:
import pandas as pd
import numpy as np

In [3]:
def ft_to_m(feet):
    return feet * 0.3048


In [4]:
runways = pd.read_csv('runways.csv')


In [5]:
runways.head()

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,le_longitude_deg,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft
0,269408,6523,00A,80.0,80.0,ASPH-G,1,0,H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,255155,6524,00AK,2500.0,70.0,GRVL,0,0,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN
2,254165,6525,00AL,2300.0,200.0,TURF,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN
3,506792,506791,00AN,4517.0,60.0,GVL,0,0,3,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN
4,322128,322127,00AS,1450.0,60.0,Turf,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN


In [6]:
runways = runways[['id', 'airport_ref', 'airport_ident', 'length_ft','lighted', 'closed']]

In [7]:
#check nulls
runways.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46840 entries, 0 to 46839
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             46840 non-null  int64  
 1   airport_ref    46840 non-null  int64  
 2   airport_ident  46840 non-null  object 
 3   length_ft      46575 non-null  float64
 4   lighted        46840 non-null  int64  
 5   closed         46840 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 2.1+ MB


there are 265 runways without length records

In [8]:
runways['airport_ident'].value_counts().head(5)

airport_ident
KORD    11
KNHU    10
JRA      9
1LA9     8
KNRQ     8
Name: count, dtype: int64

Because there are more than 1 runway for some of the airports, I want for each airport to get only the longest runway.

In [23]:
runways[runways['airport_ident'] == 'KORD']

,id,airport_ref,airport_ident,length_ft,lighted,closed
24155,245379,3754,KORD,7500.0,1,0
24156,245378,3754,KORD,8075.0,1,0
24157,341154,3754,KORD,11245.0,1,0
24158,245381,3754,KORD,7500.0,1,0
24159,245380,3754,KORD,7967.0,1,0
24160,313229,3754,KORD,10801.0,1,0
24161,250468,3754,KORD,13000.0,1,0
24162,351832,3754,KORD,7500.0,1,0
24163,245383,3754,KORD,10005.0,1,1
24164,245382,3754,KORD,9686.0,1,1


In [10]:
longest_runway = runways.groupby('airport_ident').max('length_ft')

In [11]:
longest_runway

,id,airport_ref,length_ft,lighted,closed
airport_ident,,,,,
0.00E+00,248013,7159,3270.0,0,0
00A,269408,6523,80.0,1,0
00AK,255155,6524,2500.0,0,0
00AL,254165,6525,2300.0,0,0
00AN,506792,506791,4517.0,0,0
...,...,...,...,...,...
ZYTX,235186,27243,10499.0,1,0
ZYYJ,235169,27244,8530.0,1,0
ZYYK,354997,317861,8202.0,0,0


For the nulls lengths I will inject the median length for each type of airport.
I need to get the airports.csv to check for each airport its type.

In [12]:
airports = pd.read_csv('updated_airports.csv')
types = airports[['ident', 'type']]

In [13]:
merged_with_type = pd.merge(longest_runway, types, left_on = 'airport_ident', right_on = 'ident')

In [14]:
median_runways_per_airport_type = merged_with_type[['type', 'length_ft']].groupby('type').median('length_ft')

In [15]:
longest_runway = pd.merge(merged_with_type, median_runways_per_airport_type, on = 'type')

In [16]:
longest_runway['length_ft_x'] = longest_runway['length_ft_x'].fillna(longest_runway['length_ft_y'])

In [17]:
longest_runway

,id,airport_ref,length_ft_x,lighted,closed,ident,type,length_ft_y
0,269408,6523,80.0,1,0,00A,heliport,59.0
1,255155,6524,2500.0,0,0,00AK,small_airport,2919.0
2,254165,6525,2300.0,0,0,00AL,small_airport,2919.0
3,506792,506791,4517.0,0,0,00AN,small_airport,2919.0
4,322128,322127,1450.0,0,0,00AS,small_airport,2919.0
...,...,...,...,...,...,...,...,...
39884,235186,27243,10499.0,1,0,ZYTX,large_airport,11147.0
39885,235169,27244,8530.0,1,0,ZYYJ,medium_airport,7214.0
39886,354997,317861,8202.0,0,0,ZYYK,medium_airport,7214.0
39887,346789,346788,1800.0,0,0,ZZ-0003,small_airport,2919.0


In [18]:
longest_runway.drop(['type', 'length_ft_y'], axis=1, inplace=True)

In [19]:
longest_runway['length_ft_x'] = longest_runway['length_ft_x'].apply(lambda x: ft_to_m(x))

In [20]:
longest_runway.rename(columns={'length_ft_x': 'length_meters'}, inplace=True)


In [21]:
longest_runway

,id,airport_ref,length_meters,lighted,closed,ident
0,269408,6523,24.3840,1,0,00A
1,255155,6524,762.0000,0,0,00AK
2,254165,6525,701.0400,0,0,00AL
3,506792,506791,1376.7816,0,0,00AN
4,322128,322127,441.9600,0,0,00AS
...,...,...,...,...,...,...
39884,235186,27243,3200.0952,1,0,ZYTX
39885,235169,27244,2599.9440,1,0,ZYYJ
39886,354997,317861,2499.9696,0,0,ZYYK
39887,346789,346788,548.6400,0,0,ZZ-0003


In [22]:
longest_runway.to_csv('new_runways')

Now lets make a function that fix and return as a new df with all the proccess we made

In [ ]:
def fix_runways(runways, airports):
    def ft_to_m(feet):
        return feet * 0.3048
    '''
    the func take the og runways and airports csvs and
    return the runways fixed, only the longest airstrip for each airport will
    be returned
    '''
    #filter out the non-relevant columns
    runways = runways[['id', 'airport_ref', 'airport_ident', 'length_ft','lighted', 'closed']]
    #retrieve for each airport the longest runway
    longest_runway = runways.groupby('airport_ident').max('length_ft')
    #create type-indent df, later we will use it for inject
    #the median lengths for nulls
    types = airports[['ident', 'type']]
    #merge it with our df
    merged_with_type = pd.merge(longest_runway, types, left_on = 'airport_ident', right_on = 'ident')
    #calculate the median length for each type of airport
    median_runways_per_airport_type = merged_with_type[['type', 'length_ft']].groupby('type').median('length_ft')
    #merge it with our df
    longest_runway = pd.merge(merged_with_type, median_runways_per_airport_type, on='type')
    #inject all the nulls with the median values
    longest_runway['length_ft_x'] = longest_runway['length_ft_x'].fillna(longest_runway['length_ft_y'])
    #drop unneccesary columns
    longest_runway.drop(['type', 'length_ft_y'], axis=1, inplace=True)
    #convert the lengths from ft to meters
    longest_runway['length_ft_x'] = longest_runway['length_ft_x'].apply(lambda x: ft_to_m(x))
    #rename the columns
    longest_runway.rename(columns={'length_ft_x': 'length_meters'}, inplace=True)
    return longest_runway
